# motion_basics.ipynb

このノートブックでは、位置・速度・加速度の関係を学びます。

---

## 1) 等加速度運動の例

初速度0、加速度一定の場合の位置、速度、加速度を計算し、グラフ化します。

---

## 2) 数値微分による速度・加速度の計算

位置データから速度・加速度を数値微分で求め、理論値と比較します。


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 時間の設定：0秒から10秒まで500点
t = np.linspace(0, 10, 500)

# 1) 等加速度運動の例
a = 2.0  # 加速度 [m/s^2]
v0 = 0.0  # 初速度 [m/s]
x0 = 0.0  # 初期位置 [m]

# 位置、速度、加速度の計算
x = x0 + v0 * t + 0.5 * a * t**2
v = v0 + a * t
acc = np.full_like(t, a)

plt.figure(figsize=(10,6))
plt.plot(t, x, label='位置 $x(t)$')
plt.plot(t, v, label='速度 $v(t)$')
plt.plot(t, acc, label='加速度 $a(t)$')
plt.title('等加速度運動の位置・速度・加速度')
plt.xlabel('時間 $t$ [秒]')
plt.ylabel('値')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# 2) 数値微分による速度・加速度の計算（位置データから）
dt = t[1] - t[0]

# 速度は位置の微分（前進差分）
v_num = np.gradient(x, dt)
# 加速度は速度の微分
a_num = np.gradient(v_num, dt)

plt.figure(figsize=(10,6))
plt.plot(t, a, 'k--', label='理論加速度')
plt.plot(t, a_num, 'r', label='数値微分による加速度')
plt.title('加速度の理論値と数値微分値の比較')
plt.xlabel('時間 $t$ [秒]')
plt.ylabel('加速度 [m/s^2]')
plt.legend()
plt.grid(True)
plt.show()
